In [27]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [28]:
errors_df = pd.read_csv('../../datasets/telemetry_analysis/raw/PdM_errors.csv')
failures_df = pd.read_csv('../../datasets/telemetry_analysis/raw/PdM_failures.csv')
machines_df = pd.read_csv('../../datasets/telemetry_analysis/raw/PdM_machines.csv')
maint_df = pd.read_csv('../../datasets/telemetry_analysis/raw/PdM_maint.csv')
telemetry_df = pd.read_csv('../../datasets/telemetry_analysis/raw/PdM_telemetry.csv')

df_list = [errors_df, failures_df, machines_df, maint_df, telemetry_df]

In [29]:
telemetry_df = telemetry_df.merge(errors_df, on=['datetime', 'machineID'], how='left')
# telemetry_df['error'].fillna('no error')

In [30]:
telemetry_df['errorID'] = telemetry_df['errorID'].apply(lambda x: 'error' if pd.notnull(x) else x)
telemetry_df['errorID'] = telemetry_df['errorID'].fillna('no error')

In [31]:
telemetry_df['errorID'].value_counts()

errorID
no error    872484
error         3919
Name: count, dtype: int64

In [32]:
telemetry_df.describe()

,machineID,volt,rotate,pressure,vibration
count,876403.000000,876403.000000,876403.000000,876403.000000,876403.000000
mean,50.499881,170.778661,446.581017,100.859063,40.385313
std,28.866464,15.509312,52.690553,11.049361,5.371011
min,1.000000,97.333604,138.432075,51.237106,14.877054
25%,25.000000,160.305454,412.276186,93.498263,36.777284
50%,50.000000,170.608375,447.541974,100.425337,40.237407
75%,76.000000,181.005541,482.162160,107.555370,43.785205
max,100.000000,255.124717,695.020984,185.951998,76.791072


In [33]:
scaling_features = ['volt', 'rotate', 'pressure','vibration']

for feature in scaling_features:
    telemetry_df[feature] = telemetry_df[feature]/telemetry_df[feature].max()

In [34]:
le = LabelEncoder()
telemetry_df['encoded_errors'] = le.fit_transform(telemetry_df['errorID'])

In [35]:
telemetry_df = telemetry_df.drop(columns = ["datetime", "errorID"], axis=1)

In [36]:
telemetry_df.head(10)

,machineID,volt,rotate,pressure,vibration,encoded_errors
0,1,0.690713,0.602146,0.608103,0.587147,1
1,1,0.638430,0.579475,0.513361,0.565352,1
2,1,0.670221,0.758754,0.404609,0.445089,1
3,1,0.636798,0.498042,0.587509,0.535507,1
4,1,0.617776,0.626423,0.601696,0.338457,1
5,1,0.676159,0.619152,0.515870,0.464312,1
6,1,0.613645,0.718067,0.600992,0.556756,1
7,1,0.676229,0.589370,0.543157,0.462059,1
8,1,0.687211,0.573578,0.594908,0.592286,1
9,1,0.663277,0.663074,0.563846,0.519614,1


In [37]:
print("Classes:", le.classes_)
print("Encoded values:", le.transform(le.classes_))

Classes: ['error' 'no error']
Encoded values: [0 1]


In [38]:
telemetry_df.to_csv('telemetry_analysis.csv')